In [1]:
import os
import numpy as np
import glob

import keras
from keras import models
from keras import layers
from keras import optimizers
from keras.layers import Dropout, Input
from keras.layers import Conv2D, Conv3D, Flatten, Dense, Reshape, BatchNormalization
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.layers import Conv3D, MaxPool3D, Flatten, Dense

import tensorflow as tf

Using TensorFlow backend.


In [2]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3,4"

In [3]:
train_x = np.load('/home/ckwwk/data/germination_data/old_germination_data.npy')

In [4]:
train_x.shape

(1536, 60, 60, 275, 1)

In [5]:
t1 = train_x[286:1536,:,:,20:160]
tt1 = train_x[0:286,:,:,20:160]

In [6]:
import pandas as pd

label_df = pd.read_csv("/home/ckwwk/data/germination_data/label/old_germination_label.csv")
label_df.head(5)

,label
0,0
1,0
2,0
3,1
4,0


In [7]:
from keras.utils import to_categorical
train_y = label_df[286:1536]
train_y = to_categorical(train_y)

In [8]:
test_y = label_df[0:286]
test_y = to_categorical(test_y)

In [9]:
t1 = t1.astype('float32')
tt1 = tt1.astype('float32')

In [10]:
from keras.backend import tensorflow_backend as K
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
K.set_session(tf.Session(config=config))

In [1]:
# 1x1xB인 여러 종류의 필터를 통해 파장별 convolution하여 특징을 추출하고, 4개의 layer에서 추출된 특징들을 add를 통해 더하여 좀 더 파장에 대한 특징을 파악할 수 있게 한다.

input_layer = Input((60, 60, 140, 1))

conv_layer1 = Conv3D(filters=16, kernel_size=(3, 3, 11), activation='relu',padding ='same')(input_layer)

conv_layer2_1 = Conv3D(filters=16, kernel_size=(1, 1, 1), activation='relu',padding ='same')(conv_layer1)
conv_layer2_2 = Conv3D(filters=16, kernel_size=(1, 1, 3), activation='relu',padding ='same')(conv_layer1)
conv_layer2_3 = Conv3D(filters=16, kernel_size=(1, 1, 5), activation='relu',padding ='same')(conv_layer1)
conv_layer2_4 = Conv3D(filters=16, kernel_size=(1, 1, 11), activation='relu',padding ='same')(conv_layer1)
added_1 = keras.layers.add([conv_layer2_1, conv_layer2_2,conv_layer2_3,conv_layer2_4])

pooling_layer1 = MaxPool3D(pool_size=(2, 2, 3),padding ='same')(added_1)

conv_layer3_1 = Conv3D(filters=16, kernel_size=(1, 1, 1), activation='relu',padding ='same')(pooling_layer1)
conv_layer3_2 = Conv3D(filters=16, kernel_size=(1, 1, 3), activation='relu',padding ='same')(pooling_layer1)
conv_layer3_3 = Conv3D(filters=16, kernel_size=(1, 1, 5), activation='relu',padding ='same')(pooling_layer1)
conv_layer3_4 = Conv3D(filters=16, kernel_size=(1, 1, 9), activation='relu',padding ='same')(pooling_layer1)
added_2 = keras.layers.add([conv_layer3_1, conv_layer3_2,conv_layer3_3,conv_layer3_4])

conv_layer4 = Conv3D(filters=16, kernel_size=(2, 2, 3), activation='relu',padding ='same')(added_2)

pooling_layer2 = MaxPool3D(pool_size=(2, 2, 3),padding ='same')(conv_layer4)


flatten_layer = Flatten()(pooling_layer2)

dense_layer1 = Dense(units=256, activation='relu')(flatten_layer)
dense_layer1 = Dropout(0.4)(dense_layer1)
dense_layer2 = Dense(units=128, activation='relu')(dense_layer1)
dense_layer2 = Dropout(0.4)(dense_layer2)

output_layer = Dense(units=2, activation='softmax')(dense_layer2)

## define the model with input layer and output layer
model = Model(inputs=input_layer, outputs=output_layer)

NameError: name 'Input' is not defined

In [12]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 60, 60, 140,  0                                            
__________________________________________________________________________________________________
conv3d_1 (Conv3D)               (None, 60, 60, 140,  1600        input_1[0][0]                    
__________________________________________________________________________________________________
conv3d_2 (Conv3D)               (None, 60, 60, 140,  272         conv3d_1[0][0]                   
__________________________________________________________________________________________________
conv3d_3 (Conv3D)               (None, 60, 60, 140,  784         conv3d_1[0][0]                   
____________________________________________________________________________________________

In [13]:
from keras.utils import multi_gpu_model
model = multi_gpu_model(model,gpus=2)

W1024 15:07:08.948432 139951619286848 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ck/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:190: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.



In [14]:
adam = Adam(lr=0.0001, decay=1e-06)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

W1024 15:07:09.972167 139951619286848 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ck/lib/python3.7/site-packages/keras/optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [15]:
filepath = "select_mcnn.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
tb = keras.callbacks.TensorBoard(log_dir='select_mcnn',histogram_freq=1,)
callbacks_list = [checkpoint,tb]

In [16]:
import timeit
start = timeit.default_timer()

history = model.fit(x=t1, y=train_y , batch_size=32, epochs=200,validation_split=0.1 ,callbacks=callbacks_list)

stop = timeit.default_timer()
print(stop - start)

W1024 15:07:11.143824 139951619286848 deprecation.py:323] From /home/ckwwk/.conda/envs/ck/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 1125 samples, validate on 125 samples


W1024 15:07:13.607713 139951619286848 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ck/lib/python3.7/site-packages/keras/callbacks.py:1120: The name tf.summary.histogram is deprecated. Please use tf.compat.v1.summary.histogram instead.

W1024 15:07:13.678039 139951619286848 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ck/lib/python3.7/site-packages/keras/callbacks.py:1122: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W1024 15:07:13.683113 139951619286848 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ck/lib/python3.7/site-packages/keras/callbacks.py:1125: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



Epoch 1/200
1125/1125 [==============================] - 32s 29ms/step - loss: 0.7022 - acc: 0.5600 - val_loss: 0.6546 - val_acc: 0.6480

Epoch 00001: val_acc improved from -inf to 0.64800, saving model to select_mcnn.hdf5
Epoch 2/200
1125/1125 [==============================] - 24s 21ms/step - loss: 0.6751 - acc: 0.5804 - val_loss: 0.6510 - val_acc: 0.6480

Epoch 00002: val_acc did not improve from 0.64800
Epoch 3/200
1125/1125 [==============================] - 23s 20ms/step - loss: 0.6803 - acc: 0.5796 - val_loss: 0.6527 - val_acc: 0.6480

Epoch 00003: val_acc did not improve from 0.64800
Epoch 4/200
1125/1125 [==============================] - 24s 21ms/step - loss: 0.6784 - acc: 0.5956 - val_loss: 0.6622 - val_acc: 0.6480

Epoch 00004: val_acc did not improve from 0.64800
Epoch 5/200
1125/1125 [==============================] - 25s 22ms/step - loss: 0.6757 - acc: 0.5956 - val_loss: 0.6538 - val_acc: 0.6480

Epoch 00005: val_acc did not improve from 0.64800
Epoch 6/200
1125/1125 [==

In [17]:
# load best weights
model.load_weights("select_mcnn.hdf5")
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [18]:
#model.predict(test_x,batch_size=10)
model.evaluate(tt1,test_y,batch_size=32)

286/286 [==============================] - 3s 11ms/step


[0.663110953110915, 0.6608391608391608]